In [1]:
# Load packages 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# import packages 
import re
import shutil
import os


# 1 Import total number of genes 

In [2]:
# Import total number of genes from DESeq2 code 
total = pd.read_csv("12_gene_length_distribution/total_number_of_genes.csv")
total = total.rename(columns={"x": "gene_name"})

# should be 45667 genes
#len(total["gene_name"].unique())

# 2 Import list of gene names 
with more that 10 reads in total (over all three conditions)

In [3]:
# Import genes with more than 10 reads as sum() from DESeq2 code 
genenames = pd.read_csv("12_gene_length_distribution/number_of_genes_with_total_of_10_reads.csv")
genenames = genenames.rename(columns={"x": "gene_name"})

# 2 Import annotation   

In [4]:
# import carcar_annotation_v5.gtf 
annotation = pd.read_csv('12_gene_length_distribution/carcar_annotation_v5.gtf', sep='\t', comment='#', header=None)
# renaming the header of the annotation file
annotation = annotation.rename({0: 'seqname',
                                1: 'source',
                                2: 'feature',
                                3: 'start',
                                4: 'stop',
                                5: 'score',
                                6: 'strand',
                                7: 'frame',
                                8: 'attribute'}, axis=1)

    
# define a function to extract the geneid
def extract_gene_id(attributes):
    # Using a regular expression to match and extract the desired part
    match = re.search(r'gene_id "([^"]+)"', attributes)
    if match:
        return match.group(1)
    else:
        return 0 # Return 0 if there is no match
    
# filtering the column feature for transcritps
annotation = annotation[annotation['feature'] == 'transcript']
# Apply the function to the "attributes" column and create a new column with the extracted gene ids
annotation['gene_id'] = annotation['attribute'].apply(extract_gene_id)

In [9]:
annotation

,seqname,source,feature,start,stop,score,strand,frame,attribute,gene_id,gene_length
1,scaffold_1,AUGUSTUS,transcript,246717,248723,.,-,.,"gene_id ""scaffold_1-g45517""; transcript_id ""sc...",scaffold_1-g45517,2006
9,scaffold_1,AUGUSTUS,transcript,316526,317083,.,-,.,"gene_id ""scaffold_1-g45518""; transcript_id ""sc...",scaffold_1-g45518,557
12,scaffold_1,AUGUSTUS,transcript,316526,317116,.,-,.,"gene_id ""scaffold_1-g45518""; transcript_id ""sc...",scaffold_1-g45518,590
16,scaffold_1,AUGUSTUS,transcript,317117,317526,.,-,.,"gene_id ""scaffold_1-g45519""; transcript_id ""sc...",scaffold_1-g45519,409
22,scaffold_1,AUGUSTUS,transcript,319660,320224,.,-,.,"gene_id ""scaffold_1-g45520""; transcript_id ""sc...",scaffold_1-g45520,564
...,...,...,...,...,...,...,...,...,...,...,...
2161996,scaffold_72,AUGUSTUS,transcript,43108,43483,.,-,.,"gene_id ""scaffold_72-g45760""; transcript_id ""s...",scaffold_72-g45760,375
2162002,scaffold_86,AUGUSTUS,transcript,17392,31005,.,+,.,"gene_id ""scaffold_86-g45770""; transcript_id ""s...",scaffold_86-g45770,13613
2162020,scaffold_129,AUGUSTUS,transcript,346,2361,.,-,.,"gene_id ""scaffold_129-g45565""; transcript_id ""...",scaffold_129-g45565,2015
2162028,scaffold_129,AUGUSTUS,transcript,2573,9987,.,-,.,"gene_id ""scaffold_129-g45566""; transcript_id ""...",scaffold_129-g45566,7414


# 3 Calculate gene length 

In [16]:
# calculate the gene length
annotation['gene_length'] = annotation['stop'] - annotation['start']
# export gene length as csv 
annotation[['gene_id', 'gene_length']].sort_values(by='gene_length', ascending=True).drop_duplicates(subset='gene_id', keep='last').to_csv("12_gene_length_distribution/ccar_gene_length.csv", index=False)

In [11]:
annotation['gene_length']

1           2006
9            557
12           590
16           409
22           564
           ...  
2161996      375
2162002    13613
2162020     2015
2162028     7414
2162040     3328
Name: gene_length, Length: 82557, dtype: int64

In [ ]:
# keep only the lines of the annotation df that contain the gene_id from the gene names list 
data = pd.merge(annotation, genenames, left_on='gene_id', right_on='gene_name')
# take total? 

In [ ]:
data

In [ ]:
# which genes are not unique 
data['gene_name'].value_counts()

In [ ]:
# calculate max length of gene length
data['length'] = data['stop'] - data['start']
maxima = data['length'].max()
minima = data['length'].min()
# calculate the average
mean_genelength = data['length'].mean()

In [ ]:
# PLOT: Length Distribution of Genes 
plt.figure(figsize=(10, 6))
plt.title('Length Distribution of genes')
# on the x axis: Length of genes
plt.xlabel('Length of genes')
# on the y axis: Number of genes
plt.ylabel('Number of genes')

# plot the histogram
plt.hist(data['stop'] - data['start'], bins=range(minima,maxima, 10000), color='#B2D6B1', edgecolor='black', linewidth=0.05)
plt.yscale('log')



# LABELS: 
# add label for total number of genes 
plt.text(0.95, 0.95, f'Total number of genes:\n{len(total["gene_name"])}', ha='right', va='top', transform=plt.gca().transAxes)
# add total number of unique genes 
plt.text(0.95, 0.85, f"Genes with ≥ 10 reads:\n{len(data['gene_name'].unique())}", ha='right', va='top', transform=plt.gca().transAxes)
# add numer label for average at the top right corner 
plt.text(0.95, 0.75, f'Average gene length:\n{mean_genelength:.1f}', ha='right', va='top', transform=plt.gca().transAxes)

plt.text(0.95, 0.65, f'Gene length rage:\n{minima} - {maxima}', ha='right', va='top', transform=plt.gca().transAxes)


# save the plot as png with 500 dpi
plt.savefig('12_gene_length_distribution/gene_length_distribution.png', dpi=500)
